In [ ]:
import sys
import os
sys.path.append("./../")
sys.path.append("./../external")
import numpy as np
import time
from scipy.optimize import differential_evolution, minimize
import matplotlib.pyplot as plt
from pyminion import *

from external.implementations.j2020 import j2020 
#from external.implementations.esCMAgES import esCMAgES 
#from external.implementations.AGSK import AGSK
#from external.implementations.COLSHADE import COLSHADE
import concurrent.futures
import threading

In [ ]:
class FunctionEvaluator:
    def __init__(self, func):
        self.func = func
        self.n_calls = 0

    def __call__(self, X):
        self.n_calls += 1
        ret = self.func(np.array([X]))[0]
        #print(X.shape, ret.shape)
        return ret
    
class VectorizedEvaluator : 
    def __init__(self, func):
        self.func = func
        self.n_calls = 0

    def __call__(self, X):
        self.n_calls += X.shape[0]
        ret = self.func(np.array(X))
        return ret
    
class PyminionFunc : 
    def __init__(self, func):
        self.func = func
        self.n_calls = 0

    def __call__(self, X, data=None):
        X= np.array(X)
        self.n_calls += X.shape[0]
        return  self.func(X)
    
class VectorizedEvaluatorDE : 
    def __init__(self, func):
        self.func = func
        self.n_calls = 0

    def __call__(self, X):
        X = np.array(X).T
        self.n_calls += X.shape[0]
        return self.func(X)
    
class TestFunction:
    def __init__(self, ID, fun, dim, global_minimum=None, inequality_constrains_num=0, equality_constraints_num=0):
        self.ID = ID
        self.fun = fun
        self.dimensionality = dim
        self.equality_constraints_num = equality_constraints_num
        self.inequality_constraints_num = inequality_constrains_num
        self.global_minimum = global_minimum
        self.n_calls = 0

    def evaluate(self, x):
        return self.fun(x)

    def __call__(self, x):
        x=np.array(x)
        return self.fun(x)[0], np.zeros((x.shape[0], 1)), np.zeros((x.shape[0], 1))


goptimum_cec22 = {
    1 : 300, 2: 400, 3: 600, 4: 800, 5:900, 6:1800, 7:2000, 8:2200, 9:2300, 10: 2400, 11:2600, 12: 2700
}
goptimum_cec20 = {
    1 : 100, 2: 1100, 3: 700, 4: 1900, 5:1700, 6:1600, 7:2100, 8:2200, 9:2400, 10: 2500
}

# Evaluate CEC Problem and Repeat for N times using multithreading

In [ ]:
# Global results variable
results = []
results_lock = threading.Lock()

def test_optimization_threadsafe(func, bounds, dimension, func_name, Nmaxeval):
    global results
    result = {}
    result['Dimensions'] = dimension
    result['Function'] = func_name
    # Initialize bounds
    bounds_list = [bounds] * dimension
    # Create wrapped function evaluator
    evaluator = FunctionEvaluator(func)
    vecEvaluator = VectorizedEvaluator(func)
    vecEvaluatorDE = VectorizedEvaluatorDE(func)
    evaluator_other = TestFunction(0, func, dimension, 0, 0, 0)
    pyminionFunc = PyminionFunc(func)
    popsize= int(np.ceil((np.log10(Nmaxeval))**1.8+dimension))
    #popsize = int(20+dimension/2)
    


    ##other = j2020() 
    #res = other.optimize(evaluator_other, dimension, Nmaxeval, np.full(dimension, bounds[1]),np.full(dimension, bounds[0]) )
    #result["J2020"] = res[0].objective

     #----------------------------------------------------------------#
    options= {
        "mutation_strategy": "current_to_pbest_A1_1bin",
        "memory_size": 50, 
        "archive_size_ratio": 2.6, 
        "population_reduction" : True, 
        "reduction_strategy": "linear",
        "minimum_population_size": 5, 
    }

    pyminionFunc.n_calls =0
    lshade = LSHADE (pyminionFunc, bounds_list, options=dict(options), data=None, x0=None, population_size=popsize, 
                     maxevals=Nmaxeval, tol=1e-10, callback=None, boundStrategy="reflect-random", seed=None)
    res = lshade.optimize()
    result['LSHADE'] = res.fun
    #print(pyminionFunc.n_calls)
    #----------------------------------------------------------------#

     #----------------------------------------------------------------#
    pyminionFunc.n_calls =0
    lshade_rsp = NLSHADE_RSP (pyminionFunc, bounds_list, data=None, x0=None, population_size=popsize, 
                     maxevals=Nmaxeval, tol=1e-10, callback=None, boundStrategy="reflect-random", seed=None, memory_size=50, archive_size_ratio=2.6)
    res = lshade_rsp.optimize()
    result['NLSHADE_RSP'] = res.fun
    #print(pyminionFunc.n_calls)
    #----------------------------------------------------------------#


    #----------------------------------------------------------------#
    options= {
        "mutation_strategy": "current_to_pbest_A1_1bin",
        "memory_size": 50, 
        "archive_size_ratio": 2.0, 
        "population_reduction" : True, 
        "reduction_strategy": "linear",
        "minimum_population_size": 5, 
        "refine_method":"shade"
    }
    
    pyminionFunc.n_calls=0
    arrde = ARRDE (pyminionFunc, bounds_list, options=dict(options), data=None, x0=None, population_size=popsize, 
                       maxevals=Nmaxeval, tol=1e-10, callback=None, boundStrategy="reflect-random", seed=None)
    res = arrde.optimize()
    result['ARRDE'] = res.fun
    #print(pyminionFunc.n_calls)
    #----------------------------------------------------------------#

    with results_lock:
        results.append(result)
    print(result)

Nmaxeval = 200000
dimension = 10

def run_test_optimization(j, dim):
    cec_func = CEC2022Functions(function_number=j, dimension=dim)
    test_optimization_threadsafe(cec_func, (-100, 100), dim, "func_" + str(j), Nmaxeval)


func_numbers = [1,2,3,4,5, 6, 7, 8, 9, 10, 11, 12]
#func_numbers = [2, 9, 11, 12]
Nrepeat= 10

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:

    futures = []
    for k in range(Nrepeat):
        for j in func_numbers:
            futures.append(executor.submit(run_test_optimization, j, dimension))
    concurrent.futures.wait(futures)

for num in func_numbers : 
    mydict= {}
    algoRes = {"NLSHADE_RSP" : [], "LSHADE":[], "ARRDE":[]}
    for res in list(results) : 
        for algo in algoRes.keys() : 
            if res['Function'] == "func_"+str(num) :
                algoRes[algo].append(res[algo])

    full_results= {}
    for key, val in algoRes.items() : 
        error = np.abs(np.array(val)-goptimum_cec22[num])
        #error = val
        full_results[key] = (np.min(error), np.mean(error), np.std(error))

    print("Full results for function "+str(num) +":\n\t", full_results)


In [ ]:
Nmaxeval=10000
dimension=20
popsize= int(np.ceil((np.log10(Nmaxeval))**1.8+dimension))
popsize

In [ ]:
for num in func_numbers : 
    mydict= {}
    algoRes = {"NLSHADE_RSP":[], "ARRDE":[]}
    for res in list(results) : 
        for algo in algoRes.keys() : 
            if res['Function'] == "func_"+str(num) :
                algoRes[algo].append(res[algo])

    full_results= {}
    for key, val in algoRes.items() : 
        error = np.abs(np.array(val)-goptimum_cec22[num])
        #error = val
        full_results[key] = (np.min(error), np.mean(error), np.std(error))

    print("Full results for function "+str(num) +":\n\t", full_results)

In [ ]:
for num in func_numbers : 
    mydict= {}
    algoRes = {"LSHADE":[], "ARRDE":[]}
    for res in list(results) : 
        for algo in algoRes.keys() : 
            if res['Function'] == "func_"+str(num) :
                algoRes[algo].append(res[algo])

    full_results= {}
    for key, val in algoRes.items() : 
        error = np.abs(np.array(val)-goptimum_cec22[num])
        #error = val
        full_results[key] = (np.min(error), np.mean(error), np.std(error))

    print("Full results for function "+str(num) +":\n\t", full_results)

# Simple CEC problem test 
------


In [ ]:
results = []
def test_optimization(func, bounds, dimension, func_name, Nmaxeval):
    result = {}
    result['Dimensions'] = dimension
    result['Function'] = func_name
    # Initialize bounds
    bounds_list = [bounds] * dimension
    # Create wrapped function evaluator
    evaluator = FunctionEvaluator(func)
    vecEvaluator = VectorizedEvaluator(func)
    vecEvaluatorDE = VectorizedEvaluatorDE(func)
    evaluator_other = TestFunction(0, func, dimension, 0, 0, 0)
    pyminionFunc = PyminionFunc(func)
     # Lshade Optimization
    popsize= int(np.ceil((np.log10(Nmaxeval))**1.8+dimension/2)) #20+round(dimension/2)

    other = j2020() 
    #res = other.optimize(evaluator_other, dimension, Nmaxeval, np.full(dimension, bounds[1]),np.full(dimension, bounds[0]) )
    #result["J2020"] = res[0].objective

    #----------------------------------------------------------------#
    #pyminionFunc.n_calls =0
    #de = Differential_Evolution (pyminionFunc, bounds_list, data=None, x0=None, population_size=popsize, 
    #                 maxevals=Nmaxeval, tol=1e-10, callback=None, boundStrategy="reflect-random", seed=None)
    #res = de.optimize()
    #result['DE'] = res.fun
    #print(pyminionFunc.n_calls)
    #----------------------------------------------------------------#

    #----------------------------------------------------------------#
    pyminionFunc.n_calls =0
    lshade_rsp = NLSHADE_RSP (pyminionFunc, bounds_list, data=None, x0=None, population_size=popsize, 
                     maxevals=Nmaxeval, tol=1e-10, callback=None, boundStrategy="reflect-random", seed=None, memory_size=50, archive_size_ratio=2.6)
    res = lshade_rsp.optimize()
    result['NLSHADE_RSP'] = res.fun
    print(pyminionFunc.n_calls)
    #----------------------------------------------------------------#


    #----------------------------------------------------------------#
    options= {
        "mutation_strategy": "current_to_pbest_A1_1bin",
        "memory_size": 50, 
        "archive_size_ratio": 2.6, 
        "population_reduction" : True, 
        "reduction_strategy": "linear",
        "minimum_population_size": 5, 
    }

    pyminionFunc.n_calls =0
    lshade = LSHADE (pyminionFunc, bounds_list, options=dict(options), data=None, x0=None, population_size=popsize, 
                     maxevals=Nmaxeval, tol=1e-10, callback=None, boundStrategy="reflect-random", seed=None)
    res = lshade.optimize()
    result['LSHADE'] = res.fun
    #print(pyminionFunc.n_calls)
    #----------------------------------------------------------------#


    


    #----------------------------------------------------------------#
    options= {
        "mutation_strategy": "current_to_pbest_A1_1bin",
        "memory_size": 50, 
        "archive_size_ratio": 2.6, 
        "population_reduction" : bool(True), 
        "reduction_strategy": "linear",
        "minimum_population_size": dimension, 
    }
    
    pyminionFunc.n_calls=0
    arrde = ARRDE (pyminionFunc, bounds_list, options=dict(options), data=None, x0=None, population_size=popsize, 
                       maxevals=Nmaxeval, tol=1e-10, callback=None, boundStrategy="reflect-random", seed=None)
    res = arrde.optimize()
    result["ARRDE"] = res.fun
    #print(pyminionFunc.n_calls)
    #----------------------------------------------------------------#
    
    results.append(result)
    print(result)
    
    if False:
      
      #print(len(lshade.meanCR), len(lshade.diversity))
      """
      plt.figure(figsize=(4,3))
      plt.plot(lshade.meanCR, label ="LSHADE CR")
      plt.plot(lshade.meanF, label="LSHADE F")
      plt.legend()
      plt.show()

      plt.figure(figsize=(4,3))
      plt.plot(lshade2.meanCR, label ="MFADE CR")
      plt.plot(lshade2.meanF, label="MFADE F")
      plt.legend()
      plt.show()
      
      plt.figure(figsize=(4,3))
      plt.plot(lshade.diversity, label="Diversity LSHADE")
      plt.plot(lshade2.diversity, label="Diversity LSHADE2")
      plt.plot(de.diversity, label="Diversity DE")
      plt.legend()
      plt.yscale("log")
      plt.show() 
      """
      
      plt.figure(figsize=(4,3))
      plt.plot([r.fun for r in lshade.history], label="Fitness LSHADE")
      plt.plot([r.fun for r in arrde.history], label="Fitness ARRDE")
      #plt.plot([r.fun for r in de.history], label="Fitness DE")
      plt.legend()
      plt.yscale("log")
      plt.show() 
      
      
      
Nmaxeval =2000
dimensions = [ 10]
results_list = []
for k in range(10) :
    for dim in dimensions:
        i=0
        for j in  [2, 8, 9, 10]: 
            cec_func = CEC2020Functions(function_number=j, dimension=dim)
            test_optimization(cec_func, (-100, 100), dim, "func_"+str(j), Nmaxeval)
            results_list.append(results)
            i=i+1 
        
    

In [ ]:
options = {
        "mutation_strategy": "current_to_pbest_A1_1bin",
        "memory_size": 100, 
        "archive_size_ratio": 2.6, 
        "population_reduction" : True, 
        "reduction_strategy":"linear",
        "minimum_population_size": 50, 
    }

In [ ]:
type(options)

In [ ]:
popsize= np.ceil(2*(np.log10(1e+2))**2+10/2)
popsize

-------
# Common test function
------

# 

In [ ]:
function_name_to_function = {
    "sphere": sphere,
    "rosenbrock": rosenbrock,
    "rastrigin": rastrigin,
    "drop_wave":  drop_wave, 
    "griewank": griewank,
    "ackley": ackley,
    "zakharov": zakharov,
    "goldstein_price" : goldstein_price,
    "michalewicz": michalewicz,
    "easom" : easom,
    "levy": levy,
    "dixon_price": dixon_price,
    "bent_cigar": bent_cigar,
    "discus": discus,
    "weierstrass": weierstrass,
    "happy_cat": happy_cat,
    "hgbat": hgbat,
    "hcf": hcf,
    "grie_rosen": grie_rosen,
    "escaffer6": escaffer6,
    "hybrid_composition1": hybrid_composition1,
    "hybrid_composition2": hybrid_composition2,
    "hybrid_composition3": hybrid_composition3,
    "step": step,
    "quartic": quartic,
    "schaffer2": schaffer2,
    "brown": brown,
    "exponential": exponential,
    "styblinski_tang": styblinski_tang,
    "sum_squares": sum_squares
}

# Map function implementations to function names
function_to_function_name = {v: k for k, v in function_name_to_function.items()}

# Function evaluation counter
class FunctionEvaluator:
    def __init__(self, func):
        self.func = func
        self.n_calls = 0

    def __call__(self, X):
        self.n_calls += 1
        ret = self.func(np.array([X]))[0]
        #print(X.shape, ret.shape)
        return 1000+ret
    
class VectorizedEvaluator : 
    def __init__(self, func):
        self.func = func
        self.n_calls = 0

    def __call__(self, X):
        self.n_calls += X.shape[0]
        ret = self.func(np.array(X))
        return 1000+ret
    
class PyminionFunc : 
    def __init__(self, func):
        self.func = func
        self.n_calls = 0

    def __call__(self, X, data=None):
        X= np.array(X)
        self.n_calls += X.shape[0]
        return  1000+self.func(X)
    
class VectorizedEvaluatorDE : 
    def __init__(self, func):
        self.func = func
        self.n_calls = 0

    def __call__(self, X):
        X = np.array(X).T
        self.n_calls += X.shape[0]
        return 1000+self.func(X)


In [ ]:
results = []
def test_optimization(func, bounds, dimension, func_name, Nmaxeval):
    result = {}
    result['Dimensions'] = dimension
    result['Function'] = func_name
    # Initialize bounds
    bounds_list = [bounds] * dimension
    # Create wrapped function evaluator
    evaluator = FunctionEvaluator(func)
    vecEvaluator = VectorizedEvaluator(func)
    vecEvaluatorDE = VectorizedEvaluatorDE(func)
    pyminionFunc = PyminionFunc(func)
     # Lshade Optimization
    popsize=20+round(dimension)

    lshade = LSHADE (pyminionFunc, bounds_list, data=None, x0=None, population_size=popsize, maxevals=Nmaxeval, 
                 strategy= "current_to_pbest1bin", relTol=0.0, minPopSize=max(round(dimension/2), 10), memeorySize=2*popsize, callback=None, boundStrategy="reflect-random", seed=None)
    res = lshade.optimize()
    result['LSHADE'] = res.fun


    shade = MFADE (pyminionFunc, bounds_list, data=None, x0=None, population_size=popsize, maxevals=Nmaxeval, 
                 strategy= "current_to_pbest1bin", relTol=0.0, minPopSize=max(round(dimension/2), 10), memeorySize=3*popsize, callback=None, boundStrategy="reflect-random", seed=None)
    res = shade.optimize()
    result['MFADE'] = res.fun


    jade = LJADE (pyminionFunc, bounds_list, data=None, x0=None, population_size=popsize, maxevals=Nmaxeval, 
                 strategy="current_to_pbest1bin", relTol=0.0, minPopSize=max(round(dimension/2), 10), c=0.5, callback=None, boundStrategy="reflect-random", seed=None)
    res = jade.optimize()
    result['FADE'] = res.fun

    #powell = Powell (pyminionFunc, bounds_list, data=None, x0=[1.5]*dimension, maxevals=Nmaxeval, relTol=0.0)
    #res = powell.optimize()
    #result['Powell'] = res.fun

    #powell = minimize(evaluator, x0=[1.5]*dimension, bounds=bounds_list, options={"maxfev":Nmaxeval,}, method="Powell")
    #result['Powell Scip'] = powell.fun

    #anm = NelderMead (pyminionFunc, bounds_list, data=None, x0=[1.5]*dimension, maxevals=Nmaxeval, relTol=0.0)
    #res = anm.optimize()
    #result['NM'] = res.fun

    #anm = minimize(evaluator, x0=[1.5]*dimension, bounds=bounds_list, options={"maxfev":Nmaxeval, "adaptive": True}, method="Nelder-Mead")
    #result['NM Scip'] = powell.fun

    #gwo = GWO_DE (pyminionFunc, bounds_list, data=None, x0=None, population_size=popsize, maxevals=Nmaxeval, F=0.5, CR=0.7, elimination_prob=0.1, relTol=0.0001, callback=None, boundStrategy="reflect-random", seed=None)
    #res = gwo.optimize()
    #result['GWO_DE++'] = res.fun


    #Differential Evolution (DE)
    vecEvaluatorDE.n_calls = 0
    psize = round(max(int(150/dimension), 1)*dimension)
    de_result = differential_evolution(vecEvaluatorDE, bounds_list, popsize=5, strategy='best1exp',
                                         maxiter=int(Nmaxeval/(5*dimension)), vectorized=True, updating="deferred", disp=False,polish=False)
    result['Scipy DE'] = de_result.fun


    results.append(result)
    print(result)
    

    if False:
      plt.figure(figsize=(4,3))
      plt.plot(jade.muCR, label ="FADE CR")
      plt.plot(jade.muF, label="FADE F")
      plt.legend()
      plt.show()
  

      plt.figure(figsize=(4,3))
      plt.plot(shade.muCR, label ="MFADE CR")
      plt.plot(shade.muF, label="MFADE F")
      plt.legend()
      plt.show()
     
      plt.plot([r.fun for r in jade.history], label="Fitness FADE")
      plt.plot([r.fun for r in shade.history], label="Fitness MFADE")
      plt.legend()
      plt.yscale("log")
      plt.show() 
      
Nmaxeval = 50000
dimensions = [ 20]
test = function_name_to_function
#test = {"Any" : rastrigin}
for dim in dimensions:
    i=0
    for funcname, func in test.items() : #function_name_to_function.items():
        test_optimization(func, (-100, 100), dim, funcname, Nmaxeval)
        i=i+1 
        #if i>15: break 

# Results output
import pandas as pd
pd.set_option('display.float_format', '{:.12f}'.format)
results_df = pd.DataFrame(results)
results_df